# コンピューティング先の操作

Azure Machine Learning 実験としてスクリプトを実行する場合は、実験実行の実行コンテキストを定義する必要があります。実行コンテキストは以下で構成されます。

* スクリプト向けの Python 環境。スクリプトで使用するすべての Python パッケージを含める必要があります。
* スクリプトが実行されるコンピューティング先。実験実行を開始するローカル ワークステーション、またはオンデマンドで提供されるトレーニング クラスターなどのリモート コンピューター先になります。

このノートブックでは、実験の *コンピューティング先* について学びます。

## Azure Machine Learning SDK をインストールする

Azure Machine Learning SDK は頻繁に更新されます。以下のセルを実行し、ノートブック ウィジェットをサポートする追加パッケージとともに最新のリリースにアップグレードします。

In [ ]:
!pip install --upgrade azureml-sdk azureml-widgets

## ワークスペースに接続する

最新バージョンの SDK がインストールされているため、ワークスペースに接続できます。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用データを準備する

このノートブックでは、糖尿病患者の詳細を含むデータセットを使用します。次のセルを実行してこのデータセットを作成します (すでに存在する場合は、コードによって既存のバージョンを検索します)

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                        target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                        overwrite=True, # 同じ名前の既存のファイルを置き換える
                        show_progress=True)

    #データストア上のパスから表形式のデータセットを作成する (しばらく時間がかかる場合があります)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 表形式のデータセットを登録する
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## トレーニング スクリプトを作成する

次の 2 つのセルを実行して作成します。
1. 新しい実験用のフォルダー
2. **SCIkit-learn を使用** してモデルをトレーニングし、**matplotlib** を使用して ROC 曲線をプロットするトレーニング スクリプト ファイル。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# スクリプト引数を取得する
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# 正規化ハイパーパラメーターを設定する
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データを読み込む (入力データセットとして渡される)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルのトレーニング
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 正確さを計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 曲線をプロットする
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 対角 50% ラインをプロットする
plt.plot([0, 1], [0, 1], 'k--')
# モデルによって達成された FPR と TPR をプロットする
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 環境を定義する

Azure Machine Learning で実験として Python スクリプトを実行すると、Conda 環境が作成され、スクリプトの実行コンテキストが定義されます。Azure Machine Learning には、多くの共通パッケージを含む既定の環境を提供します。これには、実験実行の操作に必要なライブラリを含む **azureml-defaults** パッケージ、**Pandas** や **numpy** などの一般なパッケージが含まれます。

また、**Conda** または **PIP** を使用して独自の環境を定義し、パッケージを追加して、実験が必要なすべてのライブラリにアクセスできるようにすることもできます。

> **注**: Conda 依存関係を最初にインストールした後、pip 依存関係をインストールします。**pip** パッケージでは pip 依存関係をインストールする必要があるため、これを Conda 依存関係にも含めるようお勧めします (忘れた場合でも Azure ML がインストールしますが、ログに警告が表示されます!)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# 実験用 Python 環境を作成する
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # 依存関係を Azure ML に管理させる
diabetes_env.docker.enabled = True # ドッカー コンテナーを使用する

# パッケージの依存関係のセットを作成する (必要に応じて Conda または PIP)
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip'],
                                             pip_packages=['azureml-sdk','pyarrow'])

# 環境に依存関係を追加する
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

# 環境を登録する
diabetes_env.register(workspace=ws)

## コンピューティング クラスターを作成する

多くの場合、ローカル コンピューティングリソースでは、大量のデータを処理する必要がある複雑な実験や長時間実行される実験を処理するには十分でない場合があります。また、クラウドでコンピューティング リソースを動的に作成して使用する機能を活用する場合もあります。Azure Machine Learning は、さまざまなコンピューティング先をサポートしており、これをワークスペースで定義し、実験の実行に使用できます。リソースの支払いは使用時にのみ行われます。

コンピューティング クラスターは、[Azure Machine Learning Studio](https://ml.azure.com) で作成するか、Azure Machine Learning SDK を使用して作成できます。以下のコード セルは指定された名前を使ってコンピューティング クラスターがあるかどうかワークスペースを確認し、ない場合は作成します。

> **重要**: 実行する前に、以下のコードで *your-compute-cluster* をコンピューティング クラスターに適した名前に変更してください。既存のクラスターがある場合はその名前を指定できます。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。英字、数字、- の文字が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 既存のコンピューティング先を確認する
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # まだ存在しない場合は、作成します
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

## リモート コンピューティングで実験を実行する

これで、作成したコンピューティング クラスターで実験を再び実行する準備が整いました。 

> **注**: コンテナー イメージは Conda 環境で構築する必要があり、スクリプトを実行する前にクラスター ノードを起動してイメージをデプロイする必要があるため、実験にはかなり時間がかかります。糖尿病トレーニング スクリプトのような簡単な実験では、これは非効率的に見えるかもしれません。しかし、数時間かかるより複雑な実験を実行する必要があると想像してください - よりスケーラブルな計算を動的に作成すると、全体の時間が大幅に短縮される可能性があります。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.widgets import RunDetails

# 登録済みの環境を取得する
registered_env = Environment.get(ws, 'diabetes-experiment-env')

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=registered_env,
                                compute_target=cluster_name) 

# 実験を送信する
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()

実験の実行を待っている間に、上のウィジェットまたは [Azure Machine Learning Studio](https://ml.azure.com) でコンピューティングの状態を確認できます。次のコマンドを使用して、コンピューティングの状態を確認することもできます。

In [ ]:
cluster_state = training_cluster.get_status()
print(cluster_state.allocation_state, cluster_state.current_node_count)

状態が *安定* から *サイズ変更中* に変わるまでにはしばらく時間がかかることに注意してください (コーヒーブレイクをするのによいタイミングです)。実行が完了するまでカーネルをブロックするには、下のセルを実行します。

In [ ]:
run.wait_for_completion()

実験が完了したら、実験の実行によって生成されたメトリックとファイルを取得できます。今回は、ファイルには、イメージを構築し、コンピューティングを管理するためのログが含まれます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

これで、実験によってトレーニングされたモデルを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Compute cluster'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**:
>
> - Azure Machine Learning でのコンピューティング先の詳細については、[Azure Machine Learning のコンピューティング先とは](https://docs.microsoft.com/azure/machine-learning/concept-compute-target)を参照してください。